# Natural Language Processing: Week 1 - Sentiment in text

NLP는 어려운 문제 중 하나이다. 수치형 형태의 픽셀 이미지를 인풋으로 받는 이미지 처리와 달리, 자연어 처리는 인풋이 수치형이 아님은 물론 문장의 길이, 순서 등 고려해야 할 것이 많다.

텍스트 처리 방법을 배우고, 감성분석 모델을 만들어 레이블이 달린 텍스트에 대한 Sentiment Analysis를 수행해보자.

## Character based encodings
먼저, 우리는 문장 집합 내 각 글자에 대한 Character Encoding을 생각해 볼 수 있다. 이러한 방법에는 대표적으로 ASCII 코드가 있는데, 여기서 각 알파벳에는 숫자가 대응된다.

다만 이러한 방식을 사용했을 때, LISTEN과 SILENT는 서로 같은 문자들의 조합으로 이루어져있기 때문에 뜻이 완전히 다름에도 불구하고 비슷한 값으로 Encoding 될 우려가 있다.

따라서 Neural Network의 입력으로 사용하기에는 부적합하다.

## Word based encodings
그렇다면 단어 기반의 Word based Encoding은 어떨까?

I love my dog 이란 문장이 있다고 가정하자. 이 문장을 단어에 기반하여 임의로 인코딩을 수행하면 다음과 같을 것이다.

$\{I:001,LOVE:002, MY:003, DOG:004\}$

그럼 I love my cat 이란 문장은 어떻게 인코딩 될까? 단어 기반 인코딩에서, 같은 단어는 항상 같은 숫자값으로 인코딩 되므로, 해당 문장의 인코딩 결과는 다음과 같을 것이다.

$\{I:001, LOVE:002, MY:003, CAT:005\}$

Cat은 I, Love, My와 달리 첫 번째 문장에서는 찾을 수 없는 단어이므로 첫 번째 문장을 구성하는 4개의 단어 중 어느 것과도 겹치지 않는 수로 인코딩 되어야 한다. 따라서, 005로 인코딩 할 수 있다.

이와 같은 단어 기반 인코딩을 적용하는 동시에, 해당 문장을 이루는 Sequence를 고려한다면, 두 문장 간의 유사도를 구하는 등의 계산을 수행할 수 있다.

단어 인코딩은 TensorFlow에서 제공하는 `Tokenizer`로 수행할 수 있으므로, 해당 API를 이용해서 진행해보도록 하자.

## Using APIs

In [ ]:
# Import Dependencies
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer

# Example Sentences
sentences = [
             'I love my dog',
             'I love my cat',
             'You love my dog!'
]

# Call Tokenizer & Create Instance
tokenizer = Tokenizer(num_words = 100) # 가장 자주 사용되는 100-1개의 단어를 고려

# Tokenize the sentences
tokenizer.fit_on_texts(sentences) # 문자 데이터를 입력받아 인코딩을 수행한 후 리스트 형태로 반환하는 메서드

# View word index
word_index = tokenizer.word_index # 단어:토큰(숫자) 쌍 딕셔너리 반환 - 단어는 모두 소문자로 변환되며 구두점은 인코딩에 영향을 주지 않는다
print(word_index)

{'love': 1, 'my': 2, 'i': 3, 'dog': 4, 'cat': 5, 'you': 6}


## Text to sequence

Tokenizer를 이용해 문장을 토큰화해봤다.

하지만 이러한 방식만으로는 부족한 점이 있다. 바로 문장의 순서이다. 문장은 단어로 이루어져있지만, 그것만으로는 부족하다. 위 예시에서 사용한 'I love my dog'은 말이 되지만, 'Love I dog my'는 말이 되지 않는다.

문장에는 단어의 순서가 있기 때문인데, 이를 우리는 Sequence라고 한다.

`texts_to_sequences()`를 이용해서 단어들을 시퀀스 형태로 변환해 보자.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer

sentences = [
  'I love my dog',
  'I love my cat',
  'You love my dog!',
  'Do you think my dog is amazing?'
]

tokenizer = Tokenizer(num_words = 100)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

# Sentences to Sequences
sequences = tokenizer.texts_to_sequences(sentences) # sentence를 token으로 이루어진 list 형태의 sequence로 변환 - 각 list를 원소로 갖는 list 형성

print(word_index)
print(sequences)

{'my': 1, 'love': 2, 'dog': 3, 'i': 4, 'you': 5, 'cat': 6, 'do': 7, 'think': 8, 'is': 9, 'amazing': 10}
[[4, 2, 1, 3], [4, 2, 1, 6], [5, 2, 1, 3], [7, 5, 8, 1, 3, 9, 10]]


위 경우, fit에 사용된 sentences 내 포함되지 않은 단어를 가진 새로운 문장이 주어졌을 때, 해당 단어는 Sequence로의 변환 과정에서 제외된다. 이를 해결할 수 있는 방법에는 두 가지가 있다.

첫 번째는 굉장히 많은 문장을 학습에 사용하는 것이고, 두 번째 방법은 **Out of Vocabulary (OOV)**를 사용하는 방법이다.

## Looking more at the Tokenizer

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

sentences = [
  'I love my dog',
  'I love my cat',
  'You love my dog!',
  'Do you think my dog is amazing?'
]

tokenizer = Tokenizer(num_words = 100, oov_token='<OOV>') # OOV Token을 추가 - 1번 인덱스 / 반드시 <OOV>로 사용할 필요는 없지만, 실제 단어와 겹치지 않을 unique한 값으로 설정하는 것이 중요
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
print(word_index)

sequences = tokenizer.texts_to_sequences(sentences)

test_sentences = [
                  'I really love my dog',
                  'My dog loves my manatee'
]

test_seq = tokenizer.texts_to_sequences(test_sentences)
print(test_seq)

{'<OOV>': 1, 'my': 2, 'love': 3, 'dog': 4, 'i': 5, 'you': 6, 'cat': 7, 'do': 8, 'think': 9, 'is': 10, 'amazing': 11}
[[5, 1, 3, 2, 4], [2, 4, 1, 2, 1]]


이미지 처리에서, 우리는 다양한 사이즈의 인풋 이미지를 하나의 사이즈로 맞춰주는 과정이 필요했다. 이것은 NLP에서도 마찬가지이며, 이를 가능하게 하는 것이 padding과 truncating이다.

## Padding

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = 100, oov_token='<OOV>')
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
print(word_index)

sequences = tokenizer.texts_to_sequences(sentences)
print(sequences)

# Padding
padded = pad_sequences(sequences) # maxlen 파라미터로 sequence의 길이를 설정해 줄 수 있다. - default값은 sentences 중 최대 길이
print(padded)

{'<OOV>': 1, 'my': 2, 'love': 3, 'dog': 4, 'i': 5, 'you': 6, 'cat': 7, 'do': 8, 'think': 9, 'is': 10, 'amazing': 11}
[[5, 3, 2, 4], [5, 3, 2, 7], [6, 3, 2, 4], [8, 6, 9, 2, 4, 10, 11]]
[[ 0  0  0  5  3  2  4]
 [ 0  0  0  5  3  2  7]
 [ 0  0  0  6  3  2  4]
 [ 8  6  9  2  4 10 11]]


만약 sentences 내 특정 sentence의 길이보다 작은 maxlen을 설정한다면, sentence의 일부는 잘리게 되고, 이것을 truncating이라고 부른다.

또한 padding/truncating 파라미터를 이용해 해당 과정의 시작 위치를 정할 수 있으며, defult 값은 pre(<-> post)이다.

In [ ]:
padded = pad_sequences(sequences, padding='post', maxlen=5)
print(padded) # 첫 번째, 두 번째 문장의 뒤에 padding이 적용되며, 세 번째 문장의 앞 쪽이 truncating 된다

[[ 5  3  2  4  0]
 [ 5  3  2  7  0]
 [ 6  3  2  4  0]
 [ 9  2  4 10 11]]


## Sarcasm, really?

- [Sarcasm in News Headlines Dataset by Rishabh Misra](https://www.kaggle.com/rmisra/news-headlines-dataset-for-sarcasm-detection)

In [ ]:
!pip install kaggle

In [ ]:
# kaggle.com > Profile: Account > API > Create New Token > kaggle.json
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kangbeenko","key":"72505225fa15a6a66bbdb1c061521ca1"}'}

In [ ]:
# Create folder name '.kaggle'
!mkdir -p ~/.kaggle

# Copy kaggle.json into .kaggle folder
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Search for the Kaggle Datasets
!kaggle datasets list -s sarcasm

ref                                                     title                                           size  lastUpdated          downloadCount  
------------------------------------------------------  ---------------------------------------------  -----  -------------------  -------------  
danofer/sarcasm                                         Sarcasm on Reddit                              216MB  2018-05-27 08:19:04           6906  
rmisra/news-headlines-dataset-for-sarcasm-detection     News Headlines Dataset For Sarcasm Detection     3MB  2019-07-03 23:52:57          27278  
saurabhbagchi/sarcasm-detection-through-nlp             Sarcasm detection                                2MB  2021-03-20 08:31:15            153  
nikhiljohnk/tweets-with-sarcasm-and-irony               Tweets with Sarcasm and Irony                    4MB  2020-11-04 10:14:04            425  
rmisra/news-category-dataset                            News Category Dataset                           25MB  2018-12-

We will use the second dataset, **'rmisra/news-headlines-dataset-for-sarcasm-detection'**

In [ ]:
# Kaggle Dataset you want > Three dots right of the 'New Notebook' > Copy API Command
!kaggle datasets download -d rmisra/news-headlines-dataset-for-sarcasm-detection

  0% 0.00/3.30M [00:00<?, ?B/s]
100% 3.30M/3.30M [00:00<00:00, 110MB/s]


In [ ]:
# Unzip the files
!unzip news-headlines-dataset-for-sarcasm-detection.zip
!ls

Archive:  news-headlines-dataset-for-sarcasm-detection.zip
  inflating: Sarcasm_Headlines_Dataset.json  
  inflating: Sarcasm_Headlines_Dataset_v2.json  
kaggle.json
news-headlines-dataset-for-sarcasm-detection.zip
sample_data
Sarcasm_Headlines_Dataset.json
Sarcasm_Headlines_Dataset_v2.json


In [ ]:
!head Sarcasm_Headlines_Dataset_v2.json

{"is_sarcastic": 1, "headline": "thirtysomething scientists unveil doomsday clock of hair loss", "article_link": "https://www.theonion.com/thirtysomething-scientists-unveil-doomsday-clock-of-hai-1819586205"}
{"is_sarcastic": 0, "headline": "dem rep. totally nails why congress is falling short on gender, racial equality", "article_link": "https://www.huffingtonpost.com/entry/donna-edwards-inequality_us_57455f7fe4b055bb1170b207"}
{"is_sarcastic": 0, "headline": "eat your veggies: 9 deliciously different recipes", "article_link": "https://www.huffingtonpost.com/entry/eat-your-veggies-9-delici_b_8899742.html"}
{"is_sarcastic": 1, "headline": "inclement weather prevents liar from getting to work", "article_link": "https://local.theonion.com/inclement-weather-prevents-liar-from-getting-to-work-1819576031"}
{"is_sarcastic": 1, "headline": "mother comes pretty close to using word 'streaming' correctly", "article_link": "https://www.theonion.com/mother-comes-pretty-close-to-using-word-streaming

This JSON File is not seperated by comma. So the Error has occured in `json.load()`

In [ ]:
!gdown --id 1xRU3xY5-tkiPGvlz5xBJ18_pHWSRzI4v

Downloading...
From: https://drive.google.com/uc?id=1xRU3xY5-tkiPGvlz5xBJ18_pHWSRzI4v
To: /content/sarcasm.json
100% 5.64M/5.64M [00:00<00:00, 88.9MB/s]


In [ ]:
!head sarcasm.json

[
{"article_link": "https://www.huffingtonpost.com/entry/versace-black-code_us_5861fbefe4b0de3a08f600d5", "headline": "former versace store clerk sues over secret 'black code' for minority shoppers", "is_sarcastic": 0},
{"article_link": "https://www.huffingtonpost.com/entry/roseanne-revival-review_us_5ab3a497e4b054d118e04365", "headline": "the 'roseanne' revival catches up to our thorny political mood, for better and worse", "is_sarcastic": 0},
{"article_link": "https://local.theonion.com/mom-starting-to-fear-son-s-web-series-closest-thing-she-1819576697", "headline": "mom starting to fear son's web series closest thing she will have to grandchild", "is_sarcastic": 1},
{"article_link": "https://politics.theonion.com/boehner-just-wants-wife-to-listen-not-come-up-with-alt-1819574302", "headline": "boehner just wants wife to listen, not come up with alternative debt-reduction ideas", "is_sarcastic": 1},
{"article_link": "https://www.huffingtonpost.com/entry/jk-rowling-wishes-snape-happy-b

In [ ]:
import json

with open('sarcasm.json', 'r') as f:
  datastore = json.load(f)

sentences = []
labels = []
urls = []

for item in datastore:
  sentences.append(item['headline'])
  labels.append(item['is_sarcastic'])
  urls.append(item['article_link'])

Sneak-peak of the Data

In [ ]:
print(sentences[0:2])
print(labels[0:2])
print(urls[0:2])

["former versace store clerk sues over secret 'black code' for minority shoppers", "the 'roseanne' revival catches up to our thorny political mood, for better and worse"]
[0, 0]
['https://www.huffingtonpost.com/entry/versace-black-code_us_5861fbefe4b0de3a08f600d5', 'https://www.huffingtonpost.com/entry/roseanne-revival-review_us_5ab3a497e4b054d118e04365']


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(sequences, padding='post')

print(sentences[0])
print(padded[0])
print(labels[0])
print(padded.shape)

former versace store clerk sues over secret 'black code' for minority shoppers
[  308 15115   679  3337  2298    48   382  2576 15116     6  2577  8434
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0]
0
(26709, 40)


Week 1 Quiz
총점 8점 / 결과 8점

###1. What is the name of the object used to tokenize sentences?
- Tokenizer
- WordTokenizer
- TextTokenizer
- CharacterTokenizer


###2. What is the name of the method used to tokenize a list of sentences?
- fit_to_text(sentences)
- tokenize_on_text(sentences)
- fit_on_texts(sentences)
- tokenize(sentences)



###3. Once you have the corpus tokenized, what’s the method used to encode a list of sentences to use those tokens?
- texts_to_tokens(sentences)
- text_to_sequences(sentences)
- texts_to_sequences(sentences)
- text_to_tokens(sentences)



###4. When initializing the tokenizer, how to you specify a token to use for unknown words?
- unknown_token=<Token>
- oov_token=<Token>
- unknown_word=<Token>
- out_of_vocab=<Token>



###5. If you don’t use a token for out of vocabulary words, what happens at encoding?
- The word isn’t encoded, and is replaced by a zero in the sequence
- The word isn’t encoded, and the sequencing ends
- The word is replaced by the most common token
- The word isn’t encoded, and is skipped in the sequence



###6. If you have a number of sequences of different lengths, how do you ensure that they are understood when fed into a neural network?
- Make sure that they are all the same length using the pad_sequences method of the tokenizer
- Use the pad_sequences object from the tensorflow.keras.preprocessing.sequence namespace
- Process them on the input layer of the Neural Netword using the pad_sequences property
- Specify the input layer of the Neural Network to expect different sizes with dynamic_length



###7. If you have a number of sequences of different length, and call pad_sequences on them, what’s the default result?
- They’ll get padded to the length of the longest sequence by adding zeros to the beginning of shorter ones
- They’ll get cropped to the length of the shortest sequence
- Nothing, they’ll remain unchanged
- They’ll get padded to the length of the longest sequence by adding zeros to the end of shorter ones


###8. When padding sequences, if you want the padding to be at the end of the sequence, how do you do it?
- Call the padding method of the pad_sequences object, passing it ‘after’
- Pass padding=’post’ to pad_sequences when initializing it
- Pass padding=’after’ to pad_sequences when initializing it
- Call the padding method of the pad_sequences object, passing it ‘post’